# 19. 基于Mindspore构造全连接层

本实验主要介绍使用MindSpore构造全连接层。并使用人为提供数据进行计算。

# 1 实验目的

- 了解全连接层的原理
- 掌握如何使用MindSpore构造全连接层

# 2 全连接层原理介绍

全连接层指的是层中的每个节点都会连接它下一层的所有节点，它是模仿人脑神经结构来构建的。用来把前边提取到的特征综合起来。由于其全相连的特性，一般全连接层的参数也是最多的。全连接层则起到将学到的“分布式特征表示”映射到样本标记空间的作用。

公式如下：
$$output=activation(X*kernel+bias)$$
其中 X 是输入Tensor， activation 是激活函数， kernel 是一个权重矩阵，其数据类型与 X 相同， bias 是一个偏置向量，其数据类型与 X 相同（仅当has_bias为True时）

具有两个隐藏层的全连接层结构如下：

![jupyter](./Figures/Fig001.png)

# 3 实验环境

在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=1.8；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore1.8 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore1.8 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore1.8 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

# 4 数据处理

## 4.1 数据准备
自生成x，x为两行三列的矩阵
$$x = [[180, 234, 154], [244, 48, 247]]$$




## 4.2 数据加载

In [1]:
from mindspore.common.tensor import Tensor  # 张量
import numpy as np                          # 科学计算库
import mindspore.common.dtype as mstype     # 数据类型模块 
# 构建一个2*3的矩阵并转换为Tensor
x = Tensor(np.array([[180, 234, 154], [244, 48, 247]]), mstype.float32)

# 5 模型构建

在使用前，导入需要的Python库，

ops模块定义了一些运算，我们需要这些API，

Cell模块是网络基类模块，我们将要构建的Dense便继承与它，

其他模块不在一一说明

详细的MindSpore的模块说明，可以在MindSpore API页面中搜索查询。



In [2]:
import mindspore.nn as nn                              # 神经网络模块
from mindspore.common.initializer import initializer   # 初始化神经元参数
from mindspore.common.parameter import Parameter       # 封装权重模块，初始化后的Parameter是Tensor的子类
from mindspore.ops import operations as P              # 常见算子操作
from mindspore._extends import cell_attr_register
from mindspore._checkparam import Rel, Validator
from mindspore.nn.cell import Cell                     # 网络基类模块
from mindspore.nn.layer.activation import get_activation    #获取激活函数模块

具体实现 Dense API

通过继承Cell类，并重写__init__方法和construct方法，来构造全连接层。

In [3]:
class Dense(Cell):
    @cell_attr_register(attrs=['has_bias', 'activation'])
    # 初始化全连接层
    def __init__(self,
                 in_channels,           #输入Tensor的空间维度
                 out_channels,          #输出Tensor的空间维度
                 weight_init='normal',  #权重参数的初始化方法，采用normal
                 bias_init='zeros',     #偏置参数的初始化方法，采用zeros
                 has_bias=True,         #是否使用偏置向量 bias，默认为True
                 activation=None        #应用于全连接层输出的激活函数，采用None
                 ):
        #调用父类初始化函数完成初始化        
        super(Dense, self).__init__()

        #in_channels、out_channels、has_bias格式检查
        self.in_channels = Validator.check_positive_int(in_channels, "in_channels", self.cls_name)
        self.out_channels = Validator.check_positive_int(out_channels, "out_channels", self.cls_name)
        self.has_bias = Validator.check_bool(has_bias, "has_bias", self.cls_name)
        
        self.reshape = P.Reshape()  #使用ops模块的Reshape，Reshape基于给定的shape，对输入Tensor进行重新排列
        self.shape_op = P.Shape()   #使用ops模块的Shape，Shape返回输入Tensor的shape
        self.weight = Parameter(initializer(weight_init, [out_channels, in_channels]), name="weight")   #初始化权重

        #定义偏置
        self.bias = None    
        if self.has_bias:
            if isinstance(bias_init, Tensor):
                if bias_init.ndim != 1 or bias_init.shape[0] != out_channels:
                    raise ValueError(f"For '{self.cls_name}', bias init shape error. The ndim of 'bias_init' must "
                                     f"be equal to 1, and the first dim must be equal to 'out_channels'. But got "
                                     f"'bias_init': {bias_init}, 'out_channels': {out_channels}.")
            self.bias = Parameter(initializer(bias_init, [out_channels]), name="bias")
            self.bias_add = P.BiasAdd()     #使用ops模块的BiasAdd

        self.matmul = P.MatMul(transpose_b=True)    #使用ops模块的MatMul

        #定义激活函数
        self.activation = get_activation(activation) if isinstance(activation, str) else activation     
        self.activation_flag = self.activation is not None

    def construct(self, x):
        x_shape = self.shape_op(x)  #获得x的shape

        if len(x_shape) != 2:       #若x的维度不是2，对x进行调整
            x = self.reshape(x, (-1, x_shape[-1]))
        x = self.matmul(x, self.weight)    #x与权重
        if self.has_bias:           #若使用偏置，对偏置与x进行相加
            x = self.bias_add(x, self.bias)
        if self.activation_flag:    #若使用激活函数，传入x进行计算
            x = self.activation(x)
        if len(x_shape) != 2:       #调整输出格式
            out_shape = x_shape[:-1] + (-1,)
            x = self.reshape(x, out_shape)
        return x

# 6 模型测试

In [4]:
# 构建输入层为3个节点，输出层为4个节点的全连接层
net = Dense(3, 4)
# 输出为2*4矩阵
output = net(x)
print(output)
print(output.shape)

[[-1.9869218  -1.7707653   2.3805907   0.07664371]
 [-0.3781587  -0.6915364   1.6991204  -2.827205  ]]
(2, 4)
